### Instalación de librerias e imports

In [1]:
!pip install igraph
!pip install cairocffi
!pip install scikit-network
!pip install tensorflow
!pip install pandas
!pip install matplotlib
!pip install scikit-learn

In [2]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.python.framework.ops import disable_eager_execution
from sklearn.decomposition import PCA
import numpy as np
from numpy import vstack
from numpy import zeros
from numpy.random import rand
from numpy.linalg import norm
from sklearn.neighbors import kneighbors_graph
from sklearn.neighbors import radius_neighbors_graph
from sklearn.neighbors import radius_neighbors_graph
from statistics import stdev
import scipy.sparse
import igraph
import matplotlib.pyplot as plt
from tensorflow.python.framework.ops import disable_eager_execution
from IPython.display import SVG
from sknetwork.data import karate_club, painters, movie_actor
from sknetwork.topology import get_connected_components, get_largest_connected_component
from sknetwork.visualization import svg_graph, svg_bigraph

ModuleNotFoundError: No module named 'sklearn.decomposition'

In [ ]:
# Necesario para ver entre las capas
disable_eager_execution()

### Función para sacar los outputs de la capas


In [ ]:
def get_layers_output(model, data):

  inp = model.input                                           
  outputs = [layer.output for layer in model.layers]          
  functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs] 

  layer_outs = [func([x, 1.]) for func in functors]
  return layer_outs

### Ejemplo para pintar grafos comentado

In [ ]:
#labels = get_connected_components(matriz_grafo)
#image = svg_graph(matriz_grafo, labels=labels)
#new_adjacency, index = get_largest_connected_component(matriz_grafo, return_index=True)
#labels = get_connected_components(matriz_grafo)
#image = svg_graph(new_adjacency, labels=index)
#new_adjacency
#index
#SVG(image)
#new_adjacency
#len(index)

## Funciones para sacar los coeficientes de enredo.

In [ ]:
def get_grafo_knn(datos,ratio_min):
  k = 1
  matriz_grafo = kneighbors_graph(np.array(datos), k, mode='distance')
  matriz_grafo = matriz_grafo.todense()
  matriz_grafo = np.array(matriz_grafo)

  for i in range(len(matriz_grafo)):
    for j in range(len(matriz_grafo)):
      matriz_grafo[j][i]=matriz_grafo[i][j]
  try:
    new_adjacency, index = get_largest_connected_component(matriz_grafo, return_index=True)
    ratio_connected = len(index)/len(datos)
  except:
    ratio_connected = 0
  if ratio_connected>ratio_min:
    matriz_grafo = new_adjacency.todense()
    g = igraph.Graph.Weighted_Adjacency(matriz_grafo, mode='undirected')
    print('Número de instancias:',len(index))
    print('número de vecinos:',k)
    return g
  g = igraph.Graph.Weighted_Adjacency(matriz_grafo, mode='undirected')
  while not g.is_connected():
    
    matriz_grafo = kneighbors_graph(datos, k, mode='distance')
    matriz_grafo = matriz_grafo.todense()
    matriz_grafo = np.array(matriz_grafo)
    
    for i in range(len(matriz_grafo)):
      for j in range(len(matriz_grafo)):
        matriz_grafo[j][i]=matriz_grafo[i][j]
    try:
      new_adjacency, index = get_largest_connected_component(matriz_grafo, return_index=True)
      ratio_connected = len(index)/len(datos)
    except:
      ratio_connected = 0
    if ratio_connected>ratio_min:
      matriz_grafo = new_adjacency.todense()
      g = igraph.Graph.Weighted_Adjacency(matriz_grafo, mode='undirected')
      print('Número de instancias:',len(index))
      print('número de vecinos:',k)
      return g
    g = igraph.Graph.Weighted_Adjacency(matriz_grafo,mode ='undirected')
    k +=1
  print('número de vecinos:', k)
  return g

In [ ]:
# Radio para que sea conectado
def radio_minimo(datos, incremento=0.01):
  n_puntos = datos.shape[0]
  G_E = distancia_euclidea(datos)
  maximo_GE = np.amax(G_E)
  diagonal_grande = np.diag(np.zeros((n_puntos,)) + maximo_GE)
  radio = np.amax(np.amin(G_E + diagonal_grande, axis=0))
  radio = radio + incremento
  return radio

In [ ]:
def get_grafo_rnn(datos,ratio_min):
  radius = 0.01
  n_puntos = datos.shape[0]
  matriz_grafo = radius_neighbors_graph(np.array(datos), radius, mode='distance')
  matriz_grafo = matriz_grafo.todense()
  g = igraph.Graph.Weighted_Adjacency(matriz_grafo, mode='undirected')

  while not g.is_connected():
    n_puntos = datos.shape[0]
    matriz_grafo = radius_neighbors_graph(datos, radius, mode='distance')
    try:
      new_adjacency, index = get_largest_connected_component(matriz_grafo, return_index=True)
      ratio_connected = len(index)/len(datos)
    except:
      ratio_connected = 0
    if ratio_connected>ratio_min:
      matriz_grafo = new_adjacency.todense()
      g = igraph.Graph.Weighted_Adjacency(matriz_grafo, mode='undirected')
      print('Número de instancias:',len(index))
      print('radio:',radius)
      return g
    matriz_grafo = matriz_grafo.todense()
    g = igraph.Graph.Weighted_Adjacency(matriz_grafo, mode='undirected')

    radius += 0.01  
  print('radio:',radius)
  return g

In [ ]:
def distancia_euclidea(datos, dim):
  #n_puntos = datos.shape[0]
  G_E = zeros((dim, dim))
  for i in range(dim):
    for j in range(dim):
      G_E[i, j] = norm(datos[i]-datos[j])
  return G_E

In [ ]:
def distancia_geodesica(datos,ratio_min,tipo):
  #radio_min = radio_minimo(datos)

  grafo = get_grafo_rnn(datos,ratio_min) if tipo == 'r' else get_grafo_knn(datos,ratio_min)
  caminos = grafo.distances(weights='weight')
  caminos = np.asarray(caminos)
  return caminos, len(caminos)

In [ ]:
def vector_triu(matriz):
  n = matriz.shape[0]
  i, j = np.triu_indices(n-1)
  return matriz[i, j+1]

In [ ]:
def coef_entangle(datos,ratio_min,tipo):
# g es el grafo correspondiente a datos
  G_M, dim = distancia_geodesica(datos,ratio_min,tipo)
  G_E = distancia_euclidea(datos, dim)
  r_E = vector_triu(G_E)
  r_M = vector_triu(G_M)
  c = np.mean(1-np.multiply(r_M - np.mean(r_M), r_E - np.mean(r_E)) / (stdev(r_E)*stdev(r_M)))
  d = np.mean(np.divide(np.abs(r_M - r_E), r_E))  
  return c, d

In [ ]:
def remove_outliers(layer):
  mu = [np.mean(l) for l in layer.transpose()]
  dev =  [np.std(l) for l in layer.transpose()]
  layer = np.array([point for point in layer if norm(point-mu)<norm(dev)])
  print('número de datos:',len(layer))
  return layer

In [ ]:
def do_report(x,layer_outputs,mask_0,mask_1,max_capas,ratio_min,tipo):
  input_0 = np.array(x.iloc[mask_0])
  input_1 = np.array(x.iloc[mask_1])
  print(f'------------ Input ------------')
  print('Variedad_0: c, d',coef_entangle(input_0,ratio_min,tipo))
  print('Variedad_1: c, d',coef_entangle(input_1,ratio_min,tipo))

  for id, layer_out in enumerate(layer_outputs[:max_capas]):
    l = list(map(list, zip(*layer_out)))
    layer_data = np.array([row[0] for row in l])
    layer_out_0 = layer_data[mask_0]
    layer_out_1 = layer_data[mask_1]
    print(f'------------CAPA {id+1}------------')
    print('Variedad_0: c, d:',coef_entangle(layer_out_0,ratio_min,tipo))
    print('Variedad_1: c, d',coef_entangle(layer_out_1,ratio_min,tipo))
  return None


In [ ]:
def do_report_table(x,layer_outputs,mask_0,mask_1,max_capas=4,ratio_min=0.6,tipo ='r'):
  input_0 = np.array(x.iloc[mask_0])
  input_1 = np.array(x.iloc[mask_1])
  df = pd.DataFrame(
   columns =['c - Clase 8', 'd-Clase 8', 'c-Clase 1', 'd-Clase 1']
)
  
  c_0,d_0 = coef_entangle(input_0,ratio_min,tipo)
  c_1,d_1 = coef_entangle(input_1,ratio_min,tipo)

  df.loc['Entrada'] = [c_0,d_0,c_1,d_1]


  for id, layer_out in enumerate(layer_outputs[:max_capas]):
    print(f'------------CAPA {id+1}------------')
    l = list(map(list, zip(*layer_out)))
    layer_data = np.array([row[0] for row in l])
    layer_out_1 = layer_data[mask_1]
    layer_out_0 = layer_data[mask_0]
    c_0,d_0 = coef_entangle(layer_out_0,ratio_min,tipo)
    c_1,d_1 = coef_entangle(layer_out_1,ratio_min,tipo)
    df.loc[f'Capa {id+1}'] = [c_0,d_0,c_1,d_1]
  return df

In [ ]:
df = pd.DataFrame(
    {
    'Medida de enredo' : ['c','d','c','d']}
)
df = df.transpose()
#df.columns= df.loc['Clase']
df.columns =['Clase 1', 'Clase 1', 'Clase 0', 'Clase 0']


# EJEMPLOS

## DATASET ESPIRAL

In [ ]:
def spiral_xy(i, spiral_num, freq):
    """
    Create the data for a spiral.
    """
    φ = math.pi*i/(freq*16)  
    r = 6.5 * ((freq*104 - i)/(freq*104))
    x = (r * math.cos(φ) * spiral_num)/13 + 0.5
    y = (r * math.sin(φ) * spiral_num)/13 + 0.5
    return (x, y)

def spiral(spiral_num):
    return [spiral_xy(i,spiral_num,1) for i in range(80)]


a_x = list(zip(*spiral(1)))[0]
a_y = list(zip(*spiral(1)))[1]

b_x = list(zip(*spiral(-1)))[0]
b_y = list(zip(*spiral(-1)))[1]

plt.figure(figsize=(8, 8))

plt.scatter(a_x,a_y, label = 'clase 1')
plt.scatter(b_x,b_y,  label = 'clase 0')
plt.legend()
plt.show()

In [ ]:
data_class_1 = [[x,y,1] for x,y in spiral(1)]
data_class_0 = [[x,y,0] for x,y in spiral(-1)]
data = data_class_1+data_class_0
data_df =pd.DataFrame(data)
data_df = data_df.sample(frac=1).reset_index(drop=True)
x, y  = data_df.iloc[:,:-1],data_df.iloc[:,-1]


In [ ]:
y

In [ ]:
x

### Entrenamiento de la red neuronal

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Enable eager execution
tf.config.run_functions_eagerly(True)

# Generate synthetic data
x = np.array([[1.764052, 0.400157],
              [0.978738, 2.240893],
              [1.867558, -0.977278],
              [0.950088, -0.151357],
              [-0.103219, 0.410599],
              [0.144044, 1.454274],
              [0.761038, 0.121675],
              [0.443863, 0.333674]])

y = np.array([0, 1, 0, 1, 0, 1, 0, 1])

# Define the model architecture
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(2,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Convert data to TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(32)

# Train the model
history = model.fit(dataset, epochs=10)

In [ ]:
accuracy = model.evaluate(x, y)[1]
layers_numbers = len(model.layers)
model_path = f'{layers_numbers}_layers_{accuracy:.2f}_accuracy.h5'
model_path

In [ ]:
model.evaluate(x, y)[1]

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/TFM Zumaquero (MAT)/9_layers_1.00_accuracy.h5')


In [ ]:
layer_outputs = get_layers_output(model, x)
mask_spiral_1 = [i==1 for i in y]
mask_spiral_0 = [i==0 for i in y]

### Reporte de las medidas de enredo aplicando el método del radio para el grafo de vecino próximo

In [ ]:
do_report(x,layer_outputs,mask_spiral_0,mask_spiral_1,0.85,'r')


In [ ]:
print(do_report_table(x,layer_outputs,mask_spiral_0,mask_spiral_1,ratio_min = 0.85, max_capas=9,tipo = 'k').to_latex())


###Reporte de las medidas de enredo aplicando k vecimos más próximos

In [ ]:
do_report(x,layer_outputs,mask_spiral_1,mask_spiral_0,0.9,'k')

## DATASET DIGITOS

In [ ]:
from sklearn.datasets import load_digits
#from keras.datasets import mnist
import pandas as pd

digits=load_digits(as_frame=True)
mnist.load_data()
X = digits.data
Y = digits.target
X['target'] = Y

X.shape

In [ ]:

digits_mask = [i==8 or i==1 for i in Y]
digits_df = X.iloc[digits_mask]
digits_df['target'] = digits_df['target'].replace(8,0)
digits_df =digits_df.reset_index(drop =True)
x, y  =  digits_df.iloc[:,:-1],digits_df.iloc[:,-1]
x.shape

#x.iloc[mask_digits_1]

### Entrenamos dataset de digitos

In [ ]:
# Fijamos la semilla para que no varie nuestro modelo de un entranamiento a otro
tf.random.set_seed(0)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
es = keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=1000, restore_best_weights=True)
model = keras.Sequential([

    layers.Dense(units= 10, activation='relu', input_shape=[64]),
    layers.Dense(units=10, activation='relu'),
    layers.Dense(units=10, activation='relu'),



    


    layers.Dense(units=1,activation = 'sigmoid'),
])
model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'],experimental_run_tf_function=False)

history = model.fit(
    x,y ,
    batch_size=60,
    epochs=300,
    callbacks = [es],
    verbose=1, # hide the output because we have so many epochs
)



In [ ]:
accuracy = model.evaluate(x, y)[1]
layers_numbers = len(model.layers)
model_path = f'bueno_digits_{layers_numbers}_layers_{accuracy:.2f}_accuracy.h5'
model_path

In [ ]:
model.save(model_path)

In [ ]:
layer_outputs_digits = get_layers_output(model, x)
mask_digits_0 = [i==0 for i in y]
mask_digits_1 = [i==1 for i in y]
sum([1 for i in mask_digits_0 if i])

### Reporte de tipo k vecinos próximos

In [ ]:
print(do_report_table(x,layer_outputs_digits,mask_digits_0,mask_digits_1,3,0.8,'k').to_latex())

### Reporte de tipo radio 

In [ ]:
do_report(x,layer_outputs_digits,mask_digits_0,mask_digits_1,0.7,'r')


### Reporte radio en formato tabla


In [ ]:
do_report_table(x,layer_outputs_digits,mask_digits_0,mask_digits_1,ratio_min = 1)
